# Vision Pipeline - Google Colab 컨트롤러

이 노트북을 사용하면 Google Colab에서 GPU를 사용하여 vision pipeline을 실행할 수 있습니다.
Google Drive를 마운트하여 저장 공간으로 사용하고 자동으로 환경을 설정합니다.

## 1. 환경 설정

In [ ]:
# @title 리포지토리 클론 및 의존성 설치
import os

# --- Configuration ---
REPO_URL = "https://github.com/ppopgi-pang/ppopgipang-vision-auto-labeler.git" # @param {type:"string"}
BRANCH = "main" # @param {type:"string"}
# ---------------------

# 리포지토리 클론
repo_name = REPO_URL.split("/")[-1].replace(".git", "")
if not os.path.exists(repo_name):
    print(f"Cloning {REPO_URL}...")
    !git clone {REPO_URL}
else:
    print(f"Repository {repo_name} already exists.")

# 프로젝트 디렉토리로 이동
%cd {repo_name}

# 의존성 설치
print("Installing dependencies...")
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt
else:
    print("requirements.txt not found, installing default dependencies...")
    # Fallback based on pyproject.toml inspection
    !pip install pyyaml pydantic pydantic-settings python-dotenv requests pillow numpy opencv-python-headless ImageHash
    !pip install torch torchvision transformers ultralytics openai playwright nest_asyncio

print("Playwright용 시스템 의존성 설치...")
!apt-get update
!apt-get install -y libatk1.0-0 libatk-bridge2.0-0 libgtk-3-0 libnss3 libx11-xcb1 \
  libxcomposite1 libxdamage1 libxrandr2 libgbm1 libasound2 libpangocairo-1.0-0 \
  libpango-1.0-0 libcups2 libdrm2 libxkbcommon0 libxfixes3 libxfext6
print("Playwright 브라우저 설치...")
!playwright install chromium

print("Done.")

## 1.1 API 키 설정
**중요:** 프로젝트 모듈을 import하기 전에 이 셀을 실행하여 키가 올바르게 로드되도록 하세요.

In [ ]:
import os

# @markdown ### API 키
# @markdown API 키를 여기에 입력하세요. 크롤링 및 LLM 검증에 필요합니다.
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# 환경 변수에 주입
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 유효성 검증 경고
if "YOUR_" in OPENAI_API_KEY:
    print("WARNING: OPENAI_API_KEY에 기본 플레이스홀더가 감지되었습니다. 실제 키로 업데이트하세요.")
    
print("환경 변수 설정 완료.")

## 2. GPU 확인

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"SUCCESS: GPU 사용 가능 - {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: GPU를 찾을 수 없습니다. Runtime > Change runtime type > Hardware accelerator > GPU에서 활성화하세요.")

## 3. Google Drive 마운트 및 경로 설정

In [ ]:
from google.colab import drive
import sys
import os

# Drive 마운트
drive.mount('/content/drive')

# 경로 설정
PROJECT_ROOT = os.getcwd()
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"Added {PROJECT_ROOT} to sys.path")

# 중요: vision_pipeline을 sys.path에 추가하여 'pipelines', 'domain' 등에서 import 가능하도록 함
PIPELINE_ROOT = os.path.join(PROJECT_ROOT, 'vision_pipeline')
if PIPELINE_ROOT not in sys.path:
    sys.path.append(PIPELINE_ROOT)
    print(f"Added {PIPELINE_ROOT} to sys.path")

# Drive에 출력 디렉토리 설정
BASE_OUTPUT_DIR = "/content/drive/MyDrive/vision_pipeline_data" # @param {type:"string"}
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)

# config.py 기본값을 오버라이드하기 위한 환경 변수 설정
os.environ["OUTPUT_DIR"] = BASE_OUTPUT_DIR
print(f"출력 디렉토리 설정: {BASE_OUTPUT_DIR}")

## 4. 파이프라인 실행

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from vision_pipeline.domain.job import Job
from vision_pipeline.services.pipeline_runner import PipelineRunner
from vision_pipeline.config import settings
import logging

# --- 실행 설정 ---
KEYWORDS = "anime character doll, anime figure doll, anime plush doll, anime character plush, anime collectible doll, anime vinyl figure, anime chibi doll, anime character toy, anime action figure, anime mascot doll, アニメ キャラクター 人形, アニメ フィギュア, アニメ ぬいぐるみ, キャラクター フィギュア, キャラクター ぬいぐるみ, 美少女 フィギュア, デフォルメ フィギュア, アニメ グッズ 人形, キャラクター ドール, フィギュア 写真, 애니 캐릭터 인형, 애니 피규어, 캐릭터 인형, 캐릭터 피규어, 애니메이션 인형, 애니 굿즈 인형, 캐릭터 굿즈 피규어, 미소녀 피규어, SD 피규어, 애니 캐릭터 장난감, anime doll figure, anime plush toy, anime figure collection, anime doll collection, anime figure photography, anime toy figure, anime doll toy, anime character merchandise doll, anime figure close up, anime doll product photo, anime figure real photo, anime doll real life, anime figure unboxing, anime figure review, anime plush doll photo, anime figure shelf, anime figure display, anime doll collection photo, anime figure product shot, anime figure desk setup, anime plush, anime stuffed doll, character plush doll, anime soft toy, anime mascot plush, anime plush collection, character plush toy, anime doll plush, anime plush figure, anime plush photo, anime scale figure, anime PVC figure, anime resin figure, anime Nendoroid, anime garage kit, anime action figure toy, anime figure model, anime collectible figure, anime statue figure, anime figure closeup, Japanese character doll, Japanese anime figure, otaku figure collection, otaku room figure, anime goods figure, anime hobby figure, anime character merchandise, anime toy collection, anime doll merchandise, anime figure goods, anime doll photo site:jp, anime figure photo site:jp, アニメ フィギュア 写真, キャラクター 人形 写真, anime plush photo site:jp, フィギュア 実物 写真, anime figure blog, anime doll review blog, フィギュア レビュー, アニメ グッズ 写真" # @param {type:"string"}
TARGET_OBJECT = "doll" # @param {type:"string"}
LIMIT = 50000 # @param {type:"integer"}

# 키워드 파싱 (공백으로 구분하거나, 선호하는 경우 단일 구문으로 처리)
# CLI는 여러 인수를 리스트로 처리. 여기서는 하나의 문자열을 받아 필요시 분할하거나
# 단순히 리스트로 래핑.
keyword_list = [k.strip() for k in KEYWORDS.split(',') if k.strip()]
if not keyword_list:
    keyword_list = [KEYWORDS]

print(f"처리 중인 키워드: {keyword_list}")
print(f"대상 객체: {TARGET_OBJECT}")
print(f"제한: {LIMIT}")
print(f"저장 경로: {settings.output_dir}")

def run_pipeline():
    # Job ID 생성
    first_kw = keyword_list[0].replace(' ', '_')
    job_id = f"job_{TARGET_OBJECT}_{first_kw}"
    
    # 기존 작업 데이터 확인하여 실수로 덮어쓰기/재실행 방지
    # (파이프라인 로직 자체가 건너뛰기를 이상적으로 처리하지만 경고)
    job_path = os.path.join(settings.output_dir, job_id)
    if os.path.exists(job_path):
        print(f"\n[INFO] 작업 디렉토리 {job_path}가 이미 존재합니다. 파이프라인이 재개하거나 완료된 단계를 건너뛰려고 시도합니다.")
    
    # Job 생성
    job = Job(
        keywords=keyword_list,
        target_class=TARGET_OBJECT,
        limit=LIMIT,
        job_id=job_id
    )
    
    # 실행
    runner = PipelineRunner()
    try:
        runner.run(job)
        print("\n대성공! 파이프라인이 완료되었습니다.")
    except Exception as e:
        print(f"\n[ERROR] 파이프라인 실패: {e}")
        logging.exception("Pipeline Failure")

# 실행
if __name__ == "__main__":
    run_pipeline()